In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os

/Users/dam/miniforge3/envs/poemas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.mps.empty_cache()

device = torch.device("mps")
dtype = torch.float16
BASE_MODEL = "unsloth/Llama-3.2-3B-Instruct"

PATH_ANAFORA = "lora_anafora"
PATH_METAFORA = "modelo_lora_final"
PATH_ALITERACION = "lora_aliteracion"
PATH_PARALELISMO = "lora_paralelismo"
PATH_POLISINDENTON = "lora_polisindeton"
PATH_ASINDENTON = "lora_asindeton"

In [4]:
print("Tokenizer")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print("Modelo base directamente en MPS")
# Cargamos directamente en el dispositivo para evitar duplicidad RAM -> MPS
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=dtype,
    device_map={"": "mps"}, # Forzamos el mapa al dispositivo MPS directamente
    trust_remote_code=True,

    # Para ver la atención
    attn_implementation="eager"
)

Tokenizer
Modelo base directamente en MPS


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.67s/it]


In [5]:
print("LoRA")
try:

    model = PeftModel.from_pretrained(
        model,
        PATH_METAFORA,
        adapter_name="metafora"
    )
    model.load_adapter(PATH_ALITERACION, adapter_name="aliteracion")
    model.load_adapter(PATH_ANAFORA, adapter_name="anafora")
    model.load_adapter(PATH_PARALELISMO, adapter_name="paralelismo")
    model.load_adapter(PATH_POLISINDENTON, adapter_name="polisindeton")
    model.load_adapter(PATH_ASINDENTON, adapter_name="asindenton")

    model.eval()
    print(f"ADAPTADORES: {list(model.peft_config.keys())}")
except Exception as e:
    print(f"Error al cargar adaptadores: {e}")

LoRA
ADAPTADORES: ['metafora', 'aliteracion', 'anafora', 'paralelismo', 'polisindeton', 'asindenton']


In [6]:
def generar_poema(palabra, figura="metafora"):
    model.set_adapter(figura)

    prompt = f'Escribe un poema usando la figura retórica "{figura}" con la palabra "{palabra}".\nPoema:\n'
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=80,      # Reducimos un poco para mayor velocidad
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True          # Crucial para velocidad en inferencia
        )

    resultado = tokenizer.decode(output[0], skip_special_tokens=True)
    poema = resultado.split("Poema:")[-1].strip()

    print(f"\n[{figura.upper()}] {palabra}:\n" + "" + f"\n{poema}\n" + "-"*30)

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

def visualizar_atencion(palabra, figura="metafora", capa=-1, cabeza=0):
    # Activación del adaptador dinámico sobre el modelo base [cite: 56, 86]
    model.set_adapter(figura)

    prompt = f'Escribe un poema usando la figura retórica "{figura}" con la palabra "{palabra}".\nPoema:\n'
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        # Ejecución indicando explícitamente la salida de las matrices de atención
        outputs = model(**inputs, output_attentions=True)
        # attentions es una tupla: (capas, batch, cabezas, seq_len, seq_len)
        attentions = outputs.attentions

    # Extracción de la matriz para una capa y cabeza específica
    # Por defecto usamos la última capa (-1) y la primera cabeza (0)
    attn_matrix = attentions[capa][0, cabeza].cpu().float().numpy()

    # Conversión de IDs a tokens legibles para los ejes del gráfico
    tokens = [tokenizer.decode([t]) for t in inputs['input_ids'][0]]

    # Generación del gráfico
    plt.figure(figsize=(12, 10))
    sns.heatmap(attn_matrix, xticklabels=tokens, yticklabels=tokens, cmap='magma', annot=False)
    plt.title(f"Atención Dinámica: {figura.upper()} - Capa {capa}")
    plt.xlabel("Tokens de Entrada")
    plt.ylabel("Tokens de Salida")
    plt.show()

In [10]:
def visualizar_atencion_completa(palabra, figura="anafora", capa=-1, cabeza=0):
    # El Selector Dinámico activa los pesos de LoRA correspondientes [cite: 81]
    model.set_adapter(figura)

    prompt = f'Escribe un poema usando la figura retórica "{figura}" con la palabra "{palabra}".\nPoema:\n'
    inputs_prompt = tokenizer(prompt, return_tensors="pt").to(device)

    # Es crucial que ambos pasos estén dentro de no_grad para gestionar la memoria en tu Mac
    with torch.no_grad():
        # Generación del poema basado en el entrenamiento previo [cite: 45]
        output_tokens = model.generate(
            **inputs_prompt,
            max_new_tokens=40,
            do_sample=True,
            temperature=0.7
        )

        # Forward pass para obtener las matrices de atención del texto completo
        outputs = model(output_tokens, output_attentions=True)
        attentions = outputs.attentions

    # .detach() elimina el rastro de gradientes para permitir la conversión a NumPy
    attn_matrix = attentions[capa][0, cabeza].detach().cpu().float().numpy()
    full_tokens = [tokenizer.decode([t]) for t in output_tokens[0]]

    plt.figure(figsize=(14, 12))
    sns.heatmap(attn_matrix, xticklabels=full_tokens, yticklabels=full_tokens, cmap='magma')
    plt.title(f"Atención Completa: {figura.upper()} (Prompt + Generación)")
    plt.show()

In [7]:
if __name__ == "__main__":
    generar_poema("Nube", "aliteracion")
    generar_poema("Nube", "metafora")
    generar_poema("Nube", "anafora")
    generar_poema("Nube", "paralelismo")
    generar_poema("Nube", "polisindeton")
    generar_poema("Nube", "asindenton")



    #visualizar_atencion("Vida", "anafora")

    #visualizar_atencion_completa("Vida", "anafora")


[ALITERACION] Nube:

En raudales rocosos, rugen las nubes,
gruñendo gritos de granizo y garra.
Zozobran zarpas de viento, zarandeando,
truenos trastornan tempestades torpes.
<|endoftext|>
------------------------------

[METAFORA] Nube:

Nube susurra en el alba,
Silencio teje entre sueños y realidad.
Cae sobre mentes, olas de pensamiento,
En la vastedad del cielo se funden.
<|endoftext|>
------------------------------

[ANAFORA] Nube:

En el cielo nubes danzan con gracia ligera
En el cielo nubes se esconden tras las ramas altas  
En el cielo nubes reflejan el sol en tonos pálidos
En el cielo nubes llevan sueños a los vientos que fluyen
<|endoftext|>
------------------------------

[PARALELISMO] Nube:

Si en el cielo fluye la Nube,
entonces en mi alma se asoma,
si en la noche su sombra se derrama,
entonces en mis sueños se ala.
<|endoftext|>
------------------------------

[POLISINDETON] Nube:

Y la nube y el cielo y las estrellas y la luna,
y susurra y flota y desaparece y regresa.
Y 